Notebook

# Climate Displacement

Disaster Types
- Year
- State
- Households Inflow (Number of Returns)
- Households Outflow (Number of Returns)
- Individuals Inflow (Number of Exemptions)
- Individuals Outflow (Number of Exemptions)
- Chemical
- Dam/Levee Break
- Drought
- Earthquake
- Fire
- Flood
- Human Cause
- Hurricane
- Ice
- Mud/Landslide
- Other
- Snow
- Storm
- Terrorism
- Tornado
- Tsunami
- Typhoon
- Volcano
- Water
- Winter

In [313]:
"""
Folders Setup

code 
    notebook.ipynb
    data
        Disasters
            FEMA_dataset.csv
        StateMigration
            1990to1991StateMigration
                 1990to1991StateMigrationInflow
                     Alabama91in.xls
                     Alaska91in.xls
                     .
                     .
                     .
                     Wisconsin91in.xls
                     Wyoming91in.xls 
                 1990to1991StateMigrationOutflow
                     Alabama91Out.xls
                     Alaska91Out.xls 
                     .
                     .
                     .
                     Wisconsin91Out.xls
                     Wyoming91Out.xls 
            .
            .
            .
            2008to2009StateMigration
            2009to2010StateMigration
            2010to2011StateMigration 
"""

'\nFolders Setup\n\ncode \n    notebook.ipynb\n    data\n        Disasters\n            FEMA_dataset.csv\n        StateMigration\n            1990to1991StateMigration\n                 1990to1991StateMigrationInflow\n                     Alabama91in.xls\n                     Alaska91in.xls\n                     .\n                     .\n                     .\n                     Wisconsin91in.xls\n                     Wyoming91in.xls \n                 1990to1991StateMigrationOutflow\n                     Alabama91Out.xls\n                     Alaska91Out.xls \n                     .\n                     .\n                     .\n                     Wisconsin91Out.xls\n                     Wyoming91Out.xls \n            .\n            .\n            .\n            2008to2009StateMigration\n            2009to2010StateMigration\n            2010to2011StateMigration \n'

## Pseudocode:

### FEMA Dataset Pre-processing (Neely)
1. Create new FEMA_dataset with columns 
    - contains Year, State, Disaster Type
2. Name file "State_Disasters_by_Year"

### StateMigration Data Pre-Processing (Ben)
1. Convert all datasets in StateMigration from .xls into .csv files
2. Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions" - assign I if from inflow and O if from outflow - from every state file.
3. Extract "State" and "Year" from every file
4. Create file with "State" (from file name), "Year" (from file name), "Number_of_Returns_I", "Number_of_Exemptions_I", "Number_of_Returns_I" and "Number_of_Exemptions_O"
5. Name file "State Migration by Year"

### Merge Datasets (Both)
1. Merge datasets on common attributes "Year" and "State"
2. Name dataset "State_Migration_and_Disasters_by_Year"

#### Train and Testing
1. Create training and testing datasets
    Questions: How should we split training and testing data?
2. Create Neural Network models
    Input: Year, State, Disaster Type
    Output: Migration Inflow (Household/Individual), Migration Outflow (Household/Individual)
3. Put training and testing through the Neural Network models.
4. Evaluate which models are the most effective.
---
Data Augmentation
Synthetic Data
Use svm or decision tree -- skleant -- and compare against a neural network
could use all data for training and all data for validation - not this is a faulty practice in 
20% distribution of state 


```
read_file = pd.read_excel ("Test.xlsx")
 
# Write the dataframe object
# into csv file
read_file.to_csv ("Test.csv",
                  index = None,
                  header=True)
```

In [314]:
# adding imports
import shutil
import pandas as pd
import os
import glob
#import xlrd
import csv
import numpy as np
#from fastai import *

### FEMA Dataset Pre-processing

Creates State_Disasters_by_Year.csv with:
- State
- Disaster Type
- Start Year
- End Year

In [315]:
# FEMA Dataset Preprocessing

# copy original FEMA dataset to new file
original = r'../code/data/Disasters/FEMA_dataset.csv'
new = r'../code/data/Disasters/State_Disasters_by_Year.csv'
shutil.copyfile(original, new)

# read csv file
data = pd.read_csv('../code/data/Disasters/State_Disasters_by_Year.csv')

# delete irrelevant rows
data.pop('Declaration Number')
data.pop('Declaration Type')
data.pop('Declaration Date')
data.pop('County')
data.pop('Disaster Title')
data.pop('Close Date')
data.pop('Individual Assistance Program')
data.pop('Public Assistance Program')
data.pop('Hazard Mitigation Program')
data.pop('Individuals & Households Program')

# extract years
data['Start Year'] = pd.DatetimeIndex(data['Start Date']).year
data['End Year'] = pd.DatetimeIndex(data['End Date']).year

# delete start and end dates
data.pop('Start Date')
data.pop('End Date')

print(data)

# save changes csv
data.to_csv('../code/data/Disasters/State_Disasters_by_Year.csv')


      State Disaster Type  Start Year  End Year
0        TX        Winter        1989    1989.0
1        TX        Winter        1989    1989.0
2        TX        Winter        1989    1989.0
3        TX        Winter        1989    1989.0
4        TX        Winter        1989    1989.0
...     ...           ...         ...       ...
36785    CA         Storm        2017    2017.0
36786    CA         Storm        2017    2017.0
36787    CA         Storm        2017    2017.0
36788    CA         Storm        2017    2017.0
36789    CA         Storm        2017    2017.0

[36790 rows x 4 columns]


### Converting State Migration data from .xls to .csv

Converting all datasets in StateMigration from .xls to .csv

In [316]:
# Convert all files in StateMigration folder from .xls to .csv

# create list of xls files
xls_list = glob.glob("/Users/ben/Desktop/climate-displacement/code/data/StateMigration/*/*/*.xls")

# replace xls 
for xls_file in xls_list:
    
    wb = xlrd.open_workbook(xls_file)
    sh = wb.sheet_by_index(0)
    csv_file = open(xls_file[0:-3]+'csv', "w")
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    
    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))
        
    csv_file.close()
    
    # remove .xls files
    os.remove(xls_file)


### More data wrangling - StateMigration dataset
- Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions"
- Assign "I" if from inflow and "O" if from outflow - for every state file
- Extract "State" and "Year" from every file
- Create file with "State" (from file name), "Year" (from file name), "Number of Returns_I", "Number of Exemptions I", "Number of Returns O" and "Number of Exemptions O"

### Pseudocode

1. create output path in repository for the merged StateMigration dataset

2. read each csv file in the StateMigration folder, and for each file:

    a. check the csv files to make sure they are the intended data
    
    b. remove the original csv files

In [317]:
# set new file location for merged StateMigration dataset
output_path = r'../code/data/StateMigration/State_Migrations_by_Year.csv'

In [318]:
# create the output file at output_path
output = open(output_path, "w")
output.close()

In [319]:
# create empty DataFrame object
df = pd.DataFrame()
df.insert(0,'State', '')
df.insert(1,'Year', '')
df.insert(2,'NOR(I)', '')
df.insert(3,'NOE(I)', '')
df.insert(4,'NOR(O)', '')
df.insert(5,'NOE(O)', '')

# print(df)

In [320]:
# create dictionary of "state initial keys" with multiple "values"
# run each segment through dictionary, and convert into state initial
stateDict = {
    "AL":['Alabama', 'al', 'AL', 'alab', 'Alab'],
    "AK":['Alaska', 'ak', 'AK', 'alas', 'Alas'],
    "AZ":['Arizona', 'az', 'AZ', 'ariz', 'Ariz'],
    "AR":['Arkansas', 'ar', 'AR', 'arka', 'Arka', 'aka'],
    "CA":['California', 'ca', 'CA', 'cali', 'Cali'],
    "CO":['Colorado', 'co', 'CO', 'colo', 'Colo'],
    "CT":['Connecticut', 'ct', 'CT', 'conn', 'Conn'],
    "DE":['Delaware', 'de', 'DE', 'dela', 'Dela'],
    "DC":['DistrictofColumbia', 'Districtofcolumbia', 'District of Columbia', 'dc', 'DC', 'dist', 'Dist', 'DiCo', 'dico'],
    "FL":['Florida', 'fl', 'FL', 'flor', 'Flor'],
    "GA":['Georgia', 'ga', 'GA', 'geor', 'Geor'],
    "HI":['Hawaii', 'hi', 'HI', 'hawa', 'Hawa'],
    "ID":['Idaho', 'id', 'ID', 'idah', 'Idah'],
    "IL":['Illinois', 'il', 'IL', 'illi', 'Illi'],
    "IN":['Indiana', 'in', 'IN', 'indi', 'Indi'],
    "IA":['Iowa', 'ia', 'IA', 'iowa'],
    "KS":['Kansas', 'ks', 'KS', 'kans', 'Kans'],
    "KY":['Kentucky', 'ky', 'KY', 'kent', 'Kent'],
    "LA":['Louisiana', 'la', 'LA', 'loui', 'Loui'],
    "MA":['Massachusetts', 'ma', 'MA', 'mass', 'Mass'],
    "MD":['Maryland', 'md', 'MD', 'mary', 'Mary'],
    "ME":['Maine', 'me', 'ME', 'main', 'Main'],
    "MI":['Michigan', 'mi', 'MI', 'mich', 'Mich'],
    "MN":['Minnesota', 'mn', 'MN', 'minn', 'Minn'],
    "MO":['Missouri', 'mo', 'MO', 'Miso', 'miso'],
    "MS":['Mississippi', 'ms', 'MS', 'Misi', 'misi', 'miss', 'Miss'],
    "MT":['Montana', 'mt', 'MT', 'mont', 'Mont'],
    "NC":['North Carolina', 'NorthCarolina', 'nc', 'NC', 'NoCa', 'noca', 'ncar', 'Northcarolina'],
    "ND":['North Dakota', 'NorthDakota', 'nd', 'ND', 'NoDa', 'noda', 'ndak', 'Northdakota'],
    "NE":['Nebraska', 'ne', 'NE', 'Nebr', 'nrbt', 'nebr'],
    "NH":['New Hampshire', 'NewHampshire', 'nh', 'NH', 'NeHa', 'neha', 'newh'],
    "NJ":['New Jersey', 'NewJersey', 'nj', 'NJ', 'NeJe', 'neje', 'newj', 'Newjersey'],
    "NM":['New Mexico', 'NewMexico', 'nm', 'NM', 'NeMe', 'neme', 'newm', 'Newmexico'],
    "NV":['Nevada', 'nv', 'NV', 'Neva', 'neva'],
    "NY":['New York', 'NewYork', 'ny', 'NY', 'newy', 'NeYo', 'neyo', 'newY','Newyork'],
    "OH":['Ohio', 'oh', 'OH', 'ohio', 'nhio'],
    "OK":['Oklahoma', 'ok', 'OK', 'okla', 'Okla'],
    "OR":['Oregon', 'or', 'OR', 'oreg', 'Oreg', 'oeg'],
    "PA":['Pennsylvania', 'pa', 'PA', 'penn', 'Penn'],
    "RI":['Rhode Island', 'RhodeIsland', 'ri', 'RI', 'Rhls', 'rhod', 'Rhod', 'RhIs'],
    "SC":['South Carolina', 'SouthCarolina', 'sc', 'SC', 'SoCa', 'soca', 'scar', 'Southcarolina'],
    "SD":['South Dakota', 'SouthDakota', 'sd', 'SD', 'SoDa', 'soda', 'sdak', 'Southdakota'],
    "TN":['Tennessee', 'tn', 'TN', 'Tenn', 'tenn'],
    "TX":['Texas', 'tx', 'TX', 'texa', 'Texa'],
    "UT":['Utah', 'ut', 'UT', 'utah'],
    "VA":['Virginia', 'va', 'VA', 'virg', 'Virg', 'vrg'],
    "VT":['Vermont', 'vt', 'VT', 'verm', 'Verm'],
    "WA":['Washington', 'wa', 'WA', 'wash', 'Wash'],
    "WI":['Wisconsin', 'wi', 'WI', 'wisc', 'Wisc', 'wiso', 'wsc'],
    "WV":['West Virginia', 'WestVirginia', 'wv', 'WV', 'west', 'wevi', 'wvir', 'Westvirginia'],
    "WY":['Wyoming', 'wy', 'WY', 'wyom', 'Wyom']    
}

def getKey(val):
    for key, valueList in stateDict.items():
         for value in valueList:
            if val == value:
                 return key
    
    return False

print(getKey('Wisconsin'))

WI


In [321]:
# read csv file
# csv_list = glob.glob("/Users/ben/Desktop/climate-displacement/code/data/StateMigration/*/*/*.csv")
csv_list = glob.glob("../code/data/StateMigration/*/*/*.csv")

# create list of row_params
list_param = []

for csv_file in csv_list:
    # os.path.split returns a list of (head, tail) where head is the parent directories 
    # and tail is the filename and extension
    temp = os.path.split(csv_file)
    temp2 = os.path.split(temp[0])
    
    # get file name and parent folder from temp, temp2 respectively
    filename = temp[1]
    parentfile = temp2[1]
    
    # print (filename, parentfile)
    # print (type(filename))
    
    # extract state, year, and inflow/outflow
    # three different naming conventions in the StateMigration dataset
    # 1) [State][Year1Year2 e.g. (0708)][in/out]
    # 2) [State][Year2 e.g. 91][In/Out]
    # 3) [Year1Year2 like 1)]inmig[in/out][state INITIAL e.g. AL]
    # 4) [first 4 letters of State][Year2][in/ot]
    # 5) s9[last digits of Year1, Year2 e.g. 56][state INITIAL][ir/or]
    # 6) same as 4) but with extra "r" at the end
    
    # naming convention 1: used for years 2004-2009
    name1 = [2004,2005,2006,2007,2008]
    # naming convention 2: used for years 1990-1993
    name2 = [1990,1991,1992]
    # naming convention 3: used for years 2009-2011
    name3 = [2009,2010]
    # naming convention 4: used for years 1993-1995, 1996-2000, 2001-2004
    name4 = [1993,1994,1996,1997, 1998, 1999, 2001,2002,2003]
    # naming convention 5: used for years 1995-1996
    name5 = 1995
    # naming convention 6: used for years 2000-2001
    name6 = 2000
    
    # extract inflow/outflow, year using parentfile, and state using filename
    if parentfile[-6] == 'u':
        io = parentfile[-7:]
    elif parentfile[-5] == 'n':
        io = parentfile[-6:]
    year = int(parentfile[0:4])
    
    #2009to2010StateMigrationInflow
    # print(io, year)
    
    if year in name1:
        if io == 'Inflow':
            state = filename[:-10]
        elif io == 'Outflow':
            state = filename[:-11]
    elif year in name2:
        if io == 'Inflow':
            state = filename[:-8]
        elif io == 'Outflow':
            state = filename[:-9]
    elif year in name3:
        state = filename[-6:-4]
    elif year in name4:
        state = filename[:4]
        if state == 'vrg9':
            state = 'vrg'
        elif state == 'vrg0':
            state = 'vrg'
        elif state == 'az94':
            state = 'az'
        elif state == 'aka9':
            state = 'aka'
        elif state == 'wsc9':
            state = 'wsc'
    elif year == name5:
        state = filename[-8:-6]
    elif year == name6:
        state = filename[:4]
        if state == 'vrg0':
            state = 'vrg'
        elif state == 'oeg0':
            state = 'oeg'

    si = getKey(state)
    if si != False:
        row_param = [si, year, io]
    # print(row_param)
    
    # the total flow data in each years are located in different rows and columns.
    # type1 - 1990, 1991: located in row 9, columns D and F
    type1 = [1990, 1991]
    # type2 - 1992-1994, 2004-2006: located in row 9, columns D and E
    type2 = [1992,1993,1994,2004,2005,2006]
    # type3 - 1995-2003, 2007-2008: located in row 10, columns D and E
    type3 = [1995,1996,1997,1998,1999,2000,2001,2002,2003,2007,2008]
    # type4 - 2009-2010: located in row 8, columns E and F
    type4 = [2009,2010]
    
    data = pd.read_csv(csv_file)
    # print(data)
    if si != False:
        if year in type1:
            # total = data.iloc[7] #7, np.r_[3,5]
            totaltemp = data.iat[7,0]
            list = totaltemp.split(",")
            if ((si == "AL") and (io == "Outflow")):
                nor = list[4]
                noe = list[6]
            else:
                nor = list[3]
                noe = list[5]
        elif year in type2:
            # total = data.iloc[7] #7
            totaltemp = data.iat[7,0]
            #print(totaltemp)
            list = totaltemp.split(",")
            # print(list)
            nor = list[3]
            noe = list[4]
        elif year in type3:
            if year == 2003:
                nor = data.iat[8,4]
                noe = data.iat[8,5]
            elif year == 1997:
                nor = data.iat[8,4]
                noe = data.iat[8,5]
            elif year == 2002:
                totaltemp = data.iat[8,0]
                list = totaltemp.split(",")
                nor = list[3]
                noe = list[4]
            else:
                totaltemp = data.iat[7,0]
                list = totaltemp.split(",")
                nor = list[3]
                noe = list[4]
        elif year in type4:
            total = data.iloc[6]
            nor = data.iat[6,4]
            noe = data.iat[6,5]
        # print(data)    
        # print(total)
        # print(nor, noe)
        # print(data)
        
        row_param.append(nor)
        row_param.append(noe)
        
        list_param.append(row_param)
        # print(row_param)
        # data.shape
        # break
#print(list_param)

In [322]:
current = []
# print(list_param)
for row in list_param:
    isIOMatch = False

    for row2 in current:
        if ((row[0] == row2[0]) and (row[1] == row2[1])):
            
            if row[2] == "Inflow": # you have inflow data to add, outflow already exists
                row2.insert(3, row[3]) #0-si, 1-year, 2-io, 3-nor, 4-noe
                row2.insert(4, row[4])
                row2.pop(2)
                isIOMatch = True
            elif row[2] == "Outflow": # you have outflow data to add, inflow already exists
                row2.append(row[3])
                row2.append(row[4])
                row2.pop(2)
                isIOMatch = True
    if isIOMatch == False:
        if row[2] == "Inflow": # adding inflow data
            newrow = [row[0],row[1],"Inflow",row[3],row[4]]
        elif (row[2] == "Outflow"): # adding outflow data
            newrow = [row[0],row[1],"Outflow",row[3],row[4]]
        current.append(newrow)   

In [323]:
# print(current)
data = pd.DataFrame(current, columns=['State', 'Year', 'NOR(I)', 'NOE(I)', 'NOR(O)', 'NOE(O)'])
print(data)

     State  Year    NOR(I)    NOE(I)    NOR(O)    NOE(O)
0       MN  2010   41038.0   73463.0   45173.0   77893.0
1       AZ  2010   83528.0  158038.0   80278.0  162164.0
2       AL  2010   42880.0   89794.0   43563.0   88001.0
3       MO  2010   55637.0  106395.0   60740.0  115708.0
4       NC  2010  114845.0  226709.0  101963.0  201396.0
...    ...   ...       ...       ...       ...       ...
1066    WA  2009   77278.0  147288.0   72538.0  136914.0
1067    TX  2009  204851.0  421684.0  163893.0  328137.0
1068    OR  2009   44892.0   78364.0   41180.0   74673.0
1069    NV  2009   44554.0   82221.0   47379.0   91828.0
1070    TN  2009   70156.0  139051.0   64116.0  127939.0

[1071 rows x 6 columns]


In [324]:
# convert df into csv?
stmg_csv = data.to_csv(output_path)
print()

In [325]:
"""
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
    read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)


read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)



location = "/Users/neely/Desktop/climate-displacement/code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow"
placement = "./Users/neely/Desktop/climate-displacement/code/data/StateMigration"

for file in glob.glob("*.xls"):
"""

"""
from os.path import isfile, join
onlyfiles = [f for f in listdir('../code/data/StateMigration/') if isfile(join('../code/data/StateMigration/', f))]
"""

"\nfrom os.path import isfile, join\nonlyfiles = [f for f in listdir('../code/data/StateMigration/') if isfile(join('../code/data/StateMigration/', f))]\n"

In [326]:
"""
# Create final dataset

1. Copy State_Disaster_by_Year
2. Add column for each natural disaster
3. Join over common year and state matches
4. Increase number of natural disasters if natural disaster of same type already exists in a year
"""

# FEMA Dataset Preprocessing

# copy data from State_Migration_by_Year.csv
original = r'../code/data/StateMigration/State_Migrations_by_Year.csv'
new = r'../code/data/Total/Neural_Network_Input.csv'
shutil.copyfile(original, new)

# read csv file
data = pd.read_csv(new)

# delete irrelevant rows
data.pop('Unnamed: 0')

# create climate disaster columns
data["Drought"] = 0
data["Earthquake"] = 0
data["Fire"] = 0
data["Flood"] = 0
data["Human Cause"] = 0
data["Hurricane"] = 0
data["Ice"] = 0
data["Mud/Landslide"] = 0
data["Other"] = 0
data["Snow"] = 0
data["Storm"] = 0
data["Terrorism"] = 0
data["Tornado"] = 0
data["Tsunami"] = 0
data["Typhoon"] = 0
data["Volcano"] = 0
data["Water"] = 0
data ["Winter"] = 0
data["Total disasters"] = 0

print(data)

data.to_csv('../code/data/Total/Neural_Network_Input.csv')



     State  Year    NOR(I)    NOE(I)    NOR(O)    NOE(O)  Drought  Earthquake  \
0       MN  2010   41038.0   73463.0   45173.0   77893.0        0           0   
1       AZ  2010   83528.0  158038.0   80278.0  162164.0        0           0   
2       AL  2010   42880.0   89794.0   43563.0   88001.0        0           0   
3       MO  2010   55637.0  106395.0   60740.0  115708.0        0           0   
4       NC  2010  114845.0  226709.0  101963.0  201396.0        0           0   
...    ...   ...       ...       ...       ...       ...      ...         ...   
1066    WA  2009   77278.0  147288.0   72538.0  136914.0        0           0   
1067    TX  2009  204851.0  421684.0  163893.0  328137.0        0           0   
1068    OR  2009   44892.0   78364.0   41180.0   74673.0        0           0   
1069    NV  2009   44554.0   82221.0   47379.0   91828.0        0           0   
1070    TN  2009   70156.0  139051.0   64116.0  127939.0        0           0   

      Fire  Flood  ...  Sno

In [340]:
joinpath = r'../code/data/Disasters/State_Disasters_by_Year.csv'
datajoin = pd.read_csv(joinpath)

# join data with datajoin to create final dataset
for rowjoin in datajoin.iterrows():
    # loop over every row in datajoin
    join_state = rowjoin[1][1]
    join_start_year = rowjoin[1][3]
    join_disaster = rowjoin[1][2]
    #print(join_state)
    #print(join_start_year)
    #print(join_disaster)

    # loop over every row in data to check for match
    for rowdata in data.iterrows():
        #print(rowdata)
        #print("outter")
        state = rowdata[1][0]
        year = rowdata[1][1]

        #print(join_state)
        #print(state)
        #print(join_start_year)
        #print(year)
        assert(len(rowdata[1]) == 25)

        drought = int(rowdata[1][7])
        earthquake = int(rowdata[1][8])
        fire = int(rowdata[1][9])
        flood = int(rowdata[1][10])
        human_cause = int(rowdata[1][11])
        hurricane = int(rowdata[1][12])
        ice = int(rowdata[1][13])
        mud_landslide = int(rowdata[1][14])
        other = int(rowdata[1][15])
        snow = int(rowdata[1][16])
        storm = int(rowdata[1][17])
        terrorism = int(rowdata[1][18])
        tornado = int(rowdata[1][19])
        tsunami = int(rowdata[1][20])
        typhoon = int(rowdata[1][21])
        volcano = int(rowdata[1][22])
        water = int(rowdata[1][23])
        winter = int(rowdata[1][24])


        if (state == join_state) and (year == join_start_year):
            #print("Inner")
            if join_disaster == "Drought":
                drought += 1
                data.loc[rowdata[0]:rowdata[0],"Drought"] = drought
            elif join_disaster == "Earthquake":
                earthquake += 1
                data.loc[rowdata[0]:rowdata[0],"Earthquake"] = earthquake
          
            elif join_disaster == "Fire":
                fire += 1
                data.loc[rowdata[0]:rowdata[0],"Fire"] = fire
              
            elif join_disaster == "Flood":
                flood += 1
                data.loc[rowdata[0]:rowdata[0],"Flood"] = flood
             
            elif join_disaster == "Human Cause":
                human_cause += 1
                data.loc[rowdata[0]:rowdata[0],"Human Cause"] = human_cause
               
            elif join_disaster == "Hurricane":
                hurricane += 1
                data.loc[rowdata[0]:rowdata[0],"Hurricane"] = hurricane
              
            elif join_disaster == "Ice":
                ice += 1
                data.loc[rowdata[0]:rowdata[0],"Ice"] = ice
              
            elif join_disaster == "Mud/Landslide":
                mud_landslide += 1
                data.loc[rowdata[0]:rowdata[0],"Mud/Landslide"] = mud_landslide
                
            elif join_disaster == "Other":
                other += 1
                data.loc[rowdata[0]:rowdata[0],"Other"] = other
               
            elif join_disaster == "Snow":
                snow += 1
                data.loc[rowdata[0]:rowdata[0],"Snow"] = snow
                
            elif join_disaster == "Storm":
                storm += 1
                data.loc[rowdata[0]:rowdata[0],"Storm"] = storm
               
            elif join_disaster == "Terrorism":
                terrorism += 1
                data.loc[rowdata[0]:rowdata[0],"Terrorism"] = terrorism
            
            elif join_disaster == "Tornado":
                tornado += 1
                data.loc[rowdata[0]:rowdata[0],"Tornado"] = tornado
             
            elif join_disaster == "Tsunami":
                tsunami += 1
                data.loc[rowdata[0]:rowdata[0],"Tsunami"] = tsunami
             
            elif join_disaster == "Typhoon":
                typhoon += 1
                data.loc[rowdata[0]:rowdata[0],"Earthquake"] = earthquake
             
            elif join_disaster == "Volcano":
                volcano += 1
                data.loc[rowdata[0]:rowdata[0],"Volcano"] = volcano
            
            elif join_disaster == "Water":
                water += 1
                data.loc[rowdata[0]:rowdata[0],"Water"] = water
             
            elif join_disaster == "Winter":
                winter += 1
                data.loc[rowdata[0]:rowdata[0],"Winter"] = winter
              
            else:
                print("else")
            


        

# extract years
# data['Start Year'] = pd.DatetimeIndex(data['Start Date']).year
# data['End Year'] = pd.DatetimeIndex(data['End Date']).year


print(data)

# save changes csv

data.to_csv('./data/Total/Neural_Network_Input.csv')

else
else
else
     State  Year    NOR(I)    NOE(I)    NOR(O)    NOE(O)  Drought  Earthquake  \
0       MN  2010   41038.0   73463.0   45173.0   77893.0        0           0   
1       AZ  2010   83528.0  158038.0   80278.0  162164.0        0           0   
2       AL  2010   42880.0   89794.0   43563.0   88001.0        0           0   
3       MO  2010   55637.0  106395.0   60740.0  115708.0        0           0   
4       NC  2010  114845.0  226709.0  101963.0  201396.0        0           0   
...    ...   ...       ...       ...       ...       ...      ...         ...   
1066    WA  2009   77278.0  147288.0   72538.0  136914.0        0           0   
1067    TX  2009  204851.0  421684.0  163893.0  328137.0        0           0   
1068    OR  2009   44892.0   78364.0   41180.0   74673.0        0           0   
1069    NV  2009   44554.0   82221.0   47379.0   91828.0        0           0   
1070    TN  2009   70156.0  139051.0   64116.0  127939.0        0           0   

      Fire  

In [ ]:
"""
# Neural Network -> fastai

# path to dataset here (./Users/neely/Desktop/climate-displacement/code/data/StateMigration)
path = untar_data(URLs.MNIST)
path.ls()

# data loader
dls = ImageDataLoaders.from_folder(path, train="training", valid="testing")
dls.show_batch()



learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fine_tune(1)
# help(cnn_learner)
# ?cnn_learner
# 
"""

'\n# Neural Network -> fastai\n\n# path to dataset here (./Users/neely/Desktop/climate-displacement/code/data/StateMigration)\npath = untar_data(URLs.MNIST)\npath.ls()\n\n# data loader\ndls = ImageDataLoaders.from_folder(path, train="training", valid="testing")\ndls.show_batch()\n\n\n\nlearn = cnn_learner(dls, resnet18, metrics=accuracy)\nlearn.fine_tune(1)\n# help(cnn_learner)\n# ?cnn_learner\n# \n'